<h1> dada2 pipeline for Taiwan reads</h1>

We're using the <a href='https://www.bioconductor.org/packages/release/bioc/html/dada2.html'>dada2</a> pipeline here. I'll try following [this tutorial](http://benjjneb.github.io/dada2/tutorial.html) Let's dive in. 

<a id='contents'></a>

[Work Environment](#installpackages)

[Getting reads ready for dada2](#prepReads)

[Demultiplexing leaf reads](#demultiplex)

[Removing primers](#removeprimers)


<a id='installpackages'></a>
<h1>Work Environment</h1>


<h3>dada2</h3>

In [1]:
source("https://bioconductor.org/biocLite.R")

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help


In [9]:
biocLite("dada2")

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.3 (2017-11-30).
Installing package(s) ‘dada2’
Warning message in install.packages(pkgs = doing, lib = lib, ...):
“installation of package ‘dada2’ had non-zero exit status”installation path not writeable, unable to update packages: codetools, lattice,
  spatial
Old packages: 'Matrix'


In [1]:
library(dada2)

Loading required package: Rcpp


In [11]:
biocLite(ask=FALSE)

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.3 (2017-11-30).
installation path not writeable, unable to update packages: codetools, lattice,
  spatial
Updating packages 'Matrix'
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘Matrix’ had non-zero exit status”Updating HTML index of packages in '.Library'
Warning message in file.create(f.tg):
“cannot create file '/usr/share/R/doc/html/packages.html', reason 'Permission denied'”Warning message in make.packages.html(.Library):
“cannot update HTML package index”

In [13]:
biocValid() 

[1] TRUE

In [2]:
packageVersion("dada2")

[1] ‘1.6.0’

<a id='prepReads'></a>
<h1>Getting reads ready for dada2</h1>

Before we begin the dada2 pipeline, we need to demultiplex our leaf reads, clean off primers and any other non-biological parts of our sequences, and make sure that forward and reverse reads are in the same order in the fastq files.  

<a id='demultiplex'></a>
<h2>Demultiplexing leaf reads</h2>

Wood reads are demultiplexed, but for the leaves are not. They were labeled using split golay barcodes, so we need to reassemble these barcodes, for both the forward and reverse reads. I have a script for this. The latest version is in this repository, BCunsplit.py

But, as I'm not much of a programmer, this script doesn't handle big read sets very well, somehow gets bogged down despite my effort to keep the entire files from being loaded into memory. So let's break up the leaf reads, apply the script, then reassemble. Most of this is in the shell, not R.

In [1]:
pwd

/home/daniel/Documents/taiwan/taiwan_dada2


In [8]:
leafR1=/home/daniel/Documents/taiwan/leafreads/TaiwanFA_R1.fastq
leafR2=/home/daniel/Documents/taiwan/leafreads/TaiwanFA_R2.fastq

In [4]:
mkdir leafR{1..2}sub

In [6]:
## break up the fastq files, keep things in multiples of four...
split -d -l 10000000 $leafR1 leafR1sub && \
split -d -l 10000000 $leafR2 leafR2sub

Now cycle through these, apply our script to each:

In [1]:
cd /home/daniel/Documents/taiwan/taiwan_dada2/leaf_recombine

In [3]:
nus=(00 01 02 03 04 05 06 07)
for i in ${nus[*]}
do
../BCunsplit.py leafR2sub$i leafR1sub$i
echo $i done!
done

00 done!
01 done!
02 done!
03 done!
04 done!
05 done!
06 done!
07 done!


Combine:

In [5]:
cat rearranged_leafR1sub0* > rearranged_leafR1.fastq && \
cat rearranged_leafR2sub0* > rearranged_leafR2.fastq 

Some sanity checks:

In [9]:
## take the first six BP of the first 10 reads from 
## the forward reads and the old... 
## check to see if they are the same
aa=$(head -n 40 rearranged_leafR2.fastq | sed -n '2~4p' | cut -c -6)
bb=$(head -n 40 $leafR2 | sed -n '2~4p' | cut -c -6)
echo $aa
echo $bb
if [ "$aa" == "$bb" ]; then echo "true"; fi

CACCTC CTTCCT CGTTAA CCTCCT CCACAT TCTCCA CCTCGC CCCCAT CCTGAT CACCTC
CACCTC CTTCCT CGTTAA CCTCCT CCACAT TCTCCA CCTCGC CCCCAT CCTGAT CACCTC
true


In [10]:
## take the first six BP of the last 10 reads from  
## the forward reads and the old
## check to see if they are the same
aa=$(tail -n 40 rearranged_leafR2.fastq | sed -n '2~4p' | cut -c -6)
bb=$(tail -n 40 $leafR2 | sed -n '2~4p' | cut -c -6)
echo $aa
echo $bb
if [ "$aa" == "$bb" ]; then echo "true"; fi

AATGAG CCCCTT AATGAT TCGCCT CCGCAT CACCTC CCTCCT CCTCAT ATAATT CTTTTN
AATGAG CCCCTT AATGAT TCGCCT CCGCAT CACCTC CCTCCT CCTCAT ATAATT CTTTTN
true


In [16]:
## take the first 12 BP of the first 10 reads from  
## the forward reads and the reverse
## check to see if they are the same
aa=$(head -n 40 rearranged_leafR2.fastq | sed -n '2~4p' | cut -c -12)
bb=$(head -n 40 rearranged_leafR1.fastq | sed -n '2~4p' | cut -c -12)
echo $aa
echo $bb
if [ "$aa" == "$bb" ]; then echo "true"; fi

CACCTCTCCTCT CTTCCTTCTTCT CGTTAACATACC CCTCCTTTCCTT CCACATCTATCT TCTCCATGCCCG CCTCGCTTCCCT CCCCATCATACC CCTGATTTCAAA CACCTCTATGAG
CACCTCTCCTCT CTTCCTTCTTCT CGTTAACATACC CCTCCTTTCCTT CCACATCTATCT TCTCCATGCCCG CCTCGCTTCCCT CCCCATCATACC CCTGATTTCAAA CACCTCTATGAG
true


Check first and last headers of the old and new reads - do the match where they need to?:

In [11]:
sed -n '1p' <(head $leafR1)
sed -n '1p' <(head rearranged_leafR1.fastq)
sed -n '1p' <(head $leafR2)
sed -n '1p' <(head rearranged_leafR2.fastq)

@HWI-M01380:62:000000000-A65GR:1:1101:12879:1376 1:N:0:
@HWI-M01380:62:000000000-A65GR:1:1101:12879:1376 1:N:0:
@HWI-M01380:62:000000000-A65GR:1:1101:12879:1376 4:N:0:
@HWI-M01380:62:000000000-A65GR:1:1101:12879:1376 4:N:0:


In [12]:
sed -n '1p' <(tail -n 4 $leafR1)
sed -n '1p' <(tail -n 4 rearranged_leafR1.fastq)
sed -n '1p' <(tail -n 4 $leafR2)
sed -n '1p' <(tail -n 4 rearranged_leafR2.fastq)

@HWI-M01380:62:000000000-A65GR:1:2119:10036:25269 1:N:0:
@HWI-M01380:62:000000000-A65GR:1:2119:10036:25269 1:N:0:
@HWI-M01380:62:000000000-A65GR:1:2119:10036:25269 4:N:0:
@HWI-M01380:62:000000000-A65GR:1:2119:10036:25269 4:N:0:


First and last headers all look good. 

Spot check that the quality scores match in length with the sequence length:

In [19]:
expr length $(sed -n '2p' <(head -n 10 rearranged_leafR2.fastq))
expr length $(sed -n '4p' <(head -n 10 rearranged_leafR2.fastq))
echo ----------------
expr length $(sed -n '2p' <(tail -n 4 rearranged_leafR2.fastq))
expr length $(sed -n '4p' <(tail -n 4 rearranged_leafR2.fastq))
echo ----------------

307
307
----------------
307
307
----------------


In [20]:
echo $(sed -n '2p' <(tail -n 4 rearranged_leafR2.fastq))
echo $(sed -n '4p' <(tail -n 4 rearranged_leafR2.fastq))
expr length $(sed -n '2p' <(tail -n 4 rearranged_leafR2.fastq))
expr length $(sed -n '4p' <(tail -n 4 rearranged_leafR2.fastq))

CTTTTNCTTTCTTNNNCTCTNNNNNNNNNNNTNNNTNNNNNTCCCCTTNNNNCNNTTCTTCTCCCTCTTTCTTCTCCTTCCCTCCTTTCTTCCCTCTTNTTCCTCTCTTCTCTCCTCTTTTCCTTCCCCTCTTCCCTTCTCCTCTTCTCCTTCTCCCCCCCCCCCTCCCCCTCCCCCTTCCTTCCCCCTCCCTTCCCCCTCCCCCTCCCCTTCTCCCTCTCTCCTCCTCCCTCCCTCCTCTCCCTTCCCTTCTTTTTCTTCCCCTCTCTTCTCCCCTCCTCTTCTTTTCTCCTTTCCCTCCCTTCTT
###################################################################################################################################################################################################################################################################################################################
307
307


Okay, sanity seems checked. 

<h3> Split the barcodes! </h3>

Fastx wants all sample names in the mapping file to be purely alphanumeric. Since we have some periods in our sample names, use sed to fix this:

In [26]:
sed -i 's/\.1/A/' leafread_fastx_map.txt && sed -i 's/\.2/B/' leafread_fastx_map.txt

Make a nest for the new files:

In [23]:
mkdir leaf_demult_fastx && mkdir leaf_demult_fastx/R{1..2}_demult 

Now use a fastx script [fastx_barcode_splitter](http://hannonlab.cshl.edu/fastx_toolkit/commandline.html#fastx_barcode_splitter_usage) to demultiplex. We'll let one barcode mismatch occur, and allow partial matches of 1 bp.

In [ ]:
cd /home/daniel/Documents/taiwan/taiwan_dada2/leaf_demult_fastx/R1_demult
cat ../../rearranged_leafR1.fastq | fastx_barcode_splitter.pl --bcfile ../../leafread_fastx_map.txt --prefix leafR1_  \
--bol --mismatches 1 --partial 1

Barcode	Count	Location
1	276173	leafR1_1
100	162293	leafR1_100
101	172051	leafR1_101
102	155374	leafR1_102
103	80347	leafR1_103
104	110477	leafR1_104
105	41854	leafR1_105
106	69	leafR1_106
107	118640	leafR1_107
108	41186	leafR1_108
109	26500	leafR1_109
110	119550	leafR1_110
111	212370	leafR1_111
112A	361	leafR1_112A
112B	18903	leafR1_112B
113A	970	leafR1_113A
113B	95	leafR1_113B
114	87023	leafR1_114
115	206173	leafR1_115
116	137356	leafR1_116
117	84449	leafR1_117
118	138033	leafR1_118
119	101369	leafR1_119
12	21711	leafR1_12
120	40949	leafR1_120
121	64312	leafR1_121
122	80257	leafR1_122
125	146097	leafR1_125
126A	93040	leafR1_126A
126B	349	leafR1_126B
127	145417	leafR1_127
128	96370	leafR1_128
129	56948	leafR1_129
13	395872	leafR1_13
130	89939	leafR1_130
131	150279	leafR1_131
132	158729	leafR1_132
133	12863	leafR1_133
14	68246	leafR1_14
15	222655	leafR1_15
16	197498	leafR1_16
17	131235	leafR1_17
18	2369	leafR1_18
19	14503	leafR1_19
2	11861	leafR1_2
20	643997	leafR1_20
21	203507	leafR1_

Repeat for R2. 

In [ ]:
cd /home/daniel/Documents/taiwan/taiwan_dada2/leaf_demult_fastx/R2_demult
cat ../../rearranged_leafR2.fastq | fastx_barcode_splitter.pl --bcfile ../../leafread_fastx_map.txt --prefix leafR2_  \
--bol --mismatches 1 --partial 1

<a id='#removeprimers'></a>
<h3>Removing Primers</h3>

Sounds simple, right? But for the leaves we have at least three types of primers to remove, I think. We have not only the classic, beginning-of-line primers (+ sample barcodes) to remove, but also two other kinds: The first is the runover - it looks like on many of these sequences, sequencing continued past the opposite primer. Then there are floating primer sequences that appear in the middle of the ITS1 region, that just don't make sense. We'll throw sequences with these floater primer sequences out entirely, because we are scared of them. 

<h3>Sequencer-runover</h3>

For instance, on the leaf R2 primers, which are our forward sequences, we often find the reverse complement of the ITS2 primer sequence, then more BPs:

In [36]:
## set some shortcut variable names:
ITS1f=CTTGGTCATTTAGAGGAAGTAA
ITS1fRC=TTACTTCCTCTAAATGACCAAG
ITS2=GCTGCGTTCTTCATCGATGC
ITS2RC=GCATCGATGAAGAACGCAGC

In [38]:
cd /home/daniel/Documents/taiwan/taiwan_dada2

## make a toy file, first 250 reads:
head -n 1000 rearranged_leafR2.fastq > toy_R2.fastq
grep -n $ITS2RC toy_R2.fastq

582:CGTTAACATACCCTTGGTNATTTAGAGGAAGTAAATGTTGCACTTTATTCAAAACAAAAGTACAAATATGCAAAGCCGAGATAAATACAACGGAGGATGAATCTACTCCAGACTTGTGGCATCGATGAAGAACGCAGCGGTATGAGATCGGAAGAGCGTCGTGTAGGGACAGAGTGTAGCTCTCGGTGCTCGTCTTATCCTTTTCTTTCTTCTCCTCCTTCCCTTCCCTCCTTCTTCCCCCTCTTCCTTTCTTCTTCCCTTCTTCCACTCCTCCCCTCCTCTCTCCTCCTCCTTCTCTTCTCTTTTC
790:AATGAGCGTCTACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTAACAATATTACATTATACACGTTTAAATTTACAAAACTAAAACTTTATGATCAATTCTGAACAAGAAATCCTTAAAACTTTCAACAACGGATCTCTTGGTTCTCGCATCGATGAAGAACGCAGCGAAATGCGATACGTACTACGAATCGCAGCTCTCGGAATCATCTCATCTTTCAACCCACTTTTCCCCAACGGGTTTTCCCCTTGGTTTTCCTGTTTTGCCCACTC


Getting rid of these should be fairly simple:

In [35]:
## cut them out with sed:
sed "s/$ITS2RC.*$//" toy_R2.fastq > toy_R2.1.fastq

## check to see if worked:
sed -n "582p" toy_R2.fastq | grep $ITS2RC
sed -n "582p" toy_R2.1.fastq 

CGTTAACATACCCTTGGTNATTTAGAGGAAGTAAATGTTGCACTTTATTCAAAACAAAAGTACAAATATGCAAAGCCGAGATAAATACAACGGAGGATGAATCTACTCCAGACTTGTGGCATCGATGAAGAACGCAGCGGTATGAGATCGGAAGAGCGTCGTGTAGGGACAGAGTGTAGCTCTCGGTGCTCGTCTTATCCTTTTCTTTCTTCTCCTCCTTCCCTTCCCTCCTTCTTCCCCCTCTTCCTTTCTTCTTCCCTTCTTCCACTCCTCCCCTCCTCTCTCCTCCTCCTTCTCTTCTCTTTTC
CGTTAACATACCCTTGGTNATTTAGAGGAAGTAAATGTTGCACTTTATTCAAAACAAAAGTACAAATATGCAAAGCCGAGATAAATACAACGGAGGATGAATCTACTCCAGACTTGTG


Works on the toy file. Apply these to our entire read sets:

In [39]:
## R1 (reverse), runover should start with ITS1fRC:
sed "s/$ITS1fRC.*$//" rearranged_leafR1.fastq > rearranged_leafR1.1.fastq

In [40]:
## R2 (forward), should have RC of ITS2 in them:
sed "s/$ITS2RC.*$//" rearranged_leafR2.fastq > rearranged_leafR2.1.fastq

<h3>Removing BOL barcodes and primers</h3>

Each sequence should begin with a 12 bp sample barcode. In addition, each read should have some BPs representing the primers (ITS1f or ITS2), this is also easy to clip. 

Barcodes are 12 bp long, ITS2 has a length of 20bp ...so... for R1 reads:

12 bp + 20 bp = 32 bp should be chopped off R1 reads. 

ITS1f has 22 bp, and there are no linker sequences in the R2 so... for R2 reads:

12 bp + 22 = 34 bp should be chopped off the R2 reads.

Does this fit? For instance, does ITS2 sit where we think it does in the R1 reads?

In [71]:
grep GCTGCGTTCTTCATCGATGC <(head -n 1000 rearranged_leafR1.1.fastq)

CACCTCTCCTCTGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGACTTATTATATTTTGACGCTCTGATTACATAAAATAACAAGCTTTTTTTGGTCCACCGGCCGCAGCTATACGACGACCTATAACTTCTTCCGACGCAACAAAAGGTATGTTCACATGGCTTGGGCGTTTAGTAAACTCTATAATGATCCCTCCGCTGGTTCACCAACGGAGTCCTTGTTACGACTTTTACTTCCTCTAACTGACCAAGCAGTTTAGATCGGTCGTCCGGTTCCGCTGCAATCC
CTTCCTTCTTCTGCTGCGTTCTTCATCGATGCCGGCGCCAAGAGATCCGTTGTTGAAAGTTTTGCTAATTTTTTACACTCACATTCGTGCCACAATCTAAGTTCTGTCAAATCCTACCGACCGCGTGGCCAATGGACCGCCCACGGCCGAACGCACGTCCTTGAAATCAGGTTTGAGGCATGGCAGTGCCAGAGGCACCGCACCAGCTAATTCTCCTTCCGCAGGTTCACCTACGGAAACCTTGTTCCGACTTTTCCTTCCTCTTAATTACCCATAGTCCCAGATCGGAAGACCGGTTCACCCGGAA
CCACATCTATCTGCTGCGTTCTTCATCGATGCCAGAACCCAGAGATCCGTTGTTAAAATTTTTGATTATTATACTTTTTCCACTCAGTAGACACGTCGTTTAATCTGAGTTTGTTTATCCTCCGGCGGGCGCCCCCCGCTGACGCGCGGCCGGGTGATGCCTGTTTCCCGACGCCGTCTTCCCCCCGATGCAACCGTTATATGTATGTTCACTACGGGTTATTTGCGTTAACTCTATAATGATCCCTCCGCTGGTTCACCACCCTCTACCTTGTTACCACTTTTACTTCCTCTAAATTCCCACTCTT
TCTCCATGCCCGGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTAAAAGTTTTGATTATTTGCTTT

CACCTCCACTGAGCTGCGTTCTTCATCGATGCCAGAGCCAAGAGATCCGTTGTTGAAAGTTTTGATTCATTTGTATTTTTGCCTTTCGGCCACTCAGAAATGCTTATAAAAACAAAGAGTTTAAGTGTCCTCGGCGGCACCGAAGCGCGCGCCGAAGCAACAAGTGGTAAGTTCACATAGGTTTTGGGTGTTGAATAACTCGATAATGATCCCTCCGCTGGTCCACCAACGGAGACCCTGTTACGACTT
CGCCATCTACAGGCTGCGTTCTTCATCGATGCCAGAACCAAGATATCCGTTGTTGAACGTTTTGTTTGATTAACTTTGACTCCGACGCAGAGATGCAGAGTTTAGATGACCTCCGGGGGCGCCACGGCAGGGTCGCCCCCGAAGCAACGAGATTTTCGTTCACACTGGGTTGGAGGTCGGTCGTGAGCCCTCACTCGGTAATTATCCCTCCGCAGGTTCACCTACGGAGACCTTGTTACTACTTTTACTTCCTCTAAATTACCAAGATGGCTAGTTCGGCAGATCCGTTCTGCAGGATTGCCAGCCC
CTAATTCATTATGCTGCGTTCTTCATCGATGCGAGAGCCAAGAGATCCGTTGGGAGGAGATTTACATAATCTGATTGTGTATATGTATTATCCTAGAAGTTATACTCAGTGTGTAAGTGCGCTCCGGCTAGCGCCGAAGCTTCTTTATGATCCTTCCGCAGGTTCACCTACGGAAACCTTGTTACGACTT
CCTGGCTCTTCTGCTGCGTTCTTCATCGATGCCTCAGGCACGCATAGCATAGTAATCGATTATCCCCTACCTAAATGCAGCCTTGGAATGGCAGCAAGGGTCGGATTACTCCTCTCTCTCCCTGTATATATAACGCATGCAGTAATTATCCAAAACTCGGATCAGTACCAGCGTAGAAAACCCCTAACCTAGTTTGGGCGCTCCTTCGCCCCGCTCGCGCCCAACCTAGCTAAGCCACCTTAGAAAAGTCT

CCGCATTATCACGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGATTCATTTTTTGTTTTAATACACTCAGACAAGCATATAAAATACAGAAGAGTTTAGGTGTCCTCCGGCGGGCGCCTGGTTCCGGGCGCGGCGTTTGTCCGCGATTCCGGGGCGAACAACCCGCCGAAGCAACGGTAAAAGGTATAAGTTCACAGTGGGTTTTGGGAGTTGTAAACTCGGTAATGATCCCTCCGCTGGTTCACCAACGGAGACCTTGTTACGACTTTTCCTTCCTCTAAATTAC
CACCTCTCTTCTGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTATAATAGTCATAGATTCTGACGCAGACTACACTACAAATAGGTTTAATAGTGGTCCTACGGACAGGCGGGCCTGCCGAGGACACGGGAGGTACTTAAAAAACATGGGTGTATATATAGTTAGGTTTGTAAGTGACAGTGCTAAGCACAACACTAGACCACGCCTAACACTATAATGTAATGATCCTTCCGCCGGTTCACCTACGGAAACCTTGTTCCGACTTTTACTTCCTCTAAATGACCCAGGC
CTCTTTTTCAAAGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTAAAAGTTTTGATTATTTGCTTGTACCACTCAGAAGAAACGTCGTTAAATCAGAGTTTGGTTATCCTCCGGCGGGCGCCGACCCGCCCGGGGGCGGGAGGCCGGGAGGGTCACGGAGACCCTACCCGCCGAAGCAACAGTTATAGGTATGTTCACAACGGGTTGTAGAGCGTAAACTCAGTAATGATCCCTCCGCTGGTTCACCAACGGAGACCTTGTTACGACTTTTACTTCCTCTAAATCACCAAGTTGGAA
CACCTCTCTTCTGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTTATTTTATTTTGC

CGTTAATATGAGGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGACTTATTTAATATAAGACTCTCAGATTACATGAAATAACATGAGTTAGGAGGTCCACCGGCGGGGCCGTTCCAGGTAACCCGTTACAGGGTAGCTACAGGGTAGGCTCCCGCCGAGGCAACCTAGGTAAGTTCACATGGTTTTGGGAGTTTTTAAAACTCTATAATGATCCCTCCGCTGGTTCACCAACGGAGACCTTGTTACGACTTTTACTTCCTCTAAATGCCCAAGTTAACTAGATCGC
CGGCGCTCTTCTGCTGCGTTCTTCATCGATGCAGAGCCAAGCGATTCGTTGTTGAAAGTTTTTGCTGTTCGCAATGTGCTCTGACCGCATTAACGTTAGGGTTCAAGGTCCTGCGCCGGGCGCGGGGCCCGGCGAGGCAACAGGTGTTTGGTAGGCAGGTAGCGGGAGGTCTGCCCCCCTGCGAGGGGGCGCCACCTTCTCTCTAATGATCCCTCCGCAGGTTCACCTACGGAGACCTTGTTACGACTT
CCCCATCTTTCTGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTAAAAGTTTTGTTTATTTTCTTTTACCTCCCACCCTAAGCTTCTTTATTTCCTTGCTTTCTTATCCTCCCGCCGTCGCCGTCCCGCCCGGATCCCCCCCGCCGCAACGATCTCTGATCCCCTACCCTCCGAAGCCACGCTTCTTCCTTTCTTCACTAATTTTTCTTCACCTTCTTCTCCTTCATTATCCTTCCTCTTCTTCTCCCTTCCTTTCCTTGTCACTCCTTTTTCTTTCTCTAACTCTCCTATCTTTGT
CCTGATCTATCTGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCATTGTTGAAAGTTTTGATGTATTTGATACGTTACTCTGAGATGCACTGGAGACGTAGTTTAGGTGTCCTCCGGCGGGCGCCCCCGA

CGTGATTAGACGGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGATTCATTTGTTTTTTACTCAGAGTTTCACTATGAAACAGAGTTTGTTTGGCCGCCGGCGCGCCGCTCCTTGTCCCCAAGGTGCCCCATAGGGACCAGCTAGCGCCGAGGCAACTGTACGTATGAGTTCACAAAGGGTTTCTGGGTGCGCCTGGGGCGCGTTCCAGCAATGATCCCTCCGCTGGTTCACCAACGGATACCTTGTTACGACTTTTCCTTCCTCTAAATGACCAAGATCACGCGCT
CATGAGCGTCTATCCTCCGCTTATTGATATGCTTACGTTCAGCGGGTAGTCCTACCTGATTTGAGGAAAAAAGTTTAGTGACTTAACACGATATTGTATAGTGTTAAATAATACTTTTAGTGACAAAACAATCCAAAGGATTAGGAGAATCCTTCGCTCAAACAGGCATACCAAGGGGAATACCCCTTGGTGCAATGTGCGTTCAAAGATTCGATGATTCCGAGAGCTGCGATTCGTATTACGTATCGCATTTCGCTGCGTTCTTCATCGATGCGAGAACCAAGAGATCCGTTGTTGAAAGTTTTAT
ACTCATTTATCCGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCATTGTTGAAAGTTTTGATTTTCCTGTTCCGTTACTCTGTGCTGCCCTGGCTACGTCGTTTAGGTGTCCTCCTGCGGGCCCCCCCGCTCCCCTGGGGGCCCGGCGACAGCCGGCGTAGCAGACCCTGGTGTGTGTATTCCTGCCGCCCGCCTTGCCCTCGGTTGTAGGGTTCACCCTTGTTCTGGGTCTTTTACTACTCGCTTATGACCCCTTTTCTTGTCCCCCCACGGCCCCCTTGTTACGACTTTTACTTCCTCTAC
CCTGCTTATGAGGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGATTCAATTTTTT

CCCCATCATACCGCTGCGTTCTTCATCGATGCCAGAGCCAAGAGATTCTTTGTTGAAAGTTGTGACTTTCTTTTACTGCGTACAGCTTCAAACAAGACTGGTCCAACGGTTATCGCCCTTTCGGGTCCATGGGCTCAGCTGAGCCGCCCACCGAGGCAACCACAGGATCGGTCAATTTGTGTGATGCAATTCCCCAGCTTACGCTGGGGTCAAGCAATGATCCCTCCGCTGTTTCACCAACGGAGCCCTTTTTACGACTTTTCCTTCCTCTAACTGACCAATCTGGTTAGATCGGCTGACCGGTTCC
CGGCGCCATACGGCTGCGTTCTTCATCGATGCCAGAGCCAAGAGATCCGTTGTTAGTAGTTTTAAATTATCTGATTGTTTATCTGTATTTTCCTAGATGTTATACTCTGTGTTTAAGTGCGCTCCGGCTAGCGCCGACGCTTCTTTATGTTCCTTCCGCAGGTTCACCTACGGACACCTTGTTCCGACTT
CCGCATCACTGAGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTAACTTATTAAGTTGAAGCATTCAGAGATACGTATATATAACAGAGTTTAGCAGTCCACCGGCGGGCCTGTAGCGCGCCGCCAGGTAGTAAACTACAGGGCGAACGCGGCCCGCCGAGGCAACGGCGGTAAGTTCACATAGGGTTGGAGTTTAGATAACTCAGTAATGATCCCTCCGCTGGTTCACCAACGGAGCCCTTGTTACGACTTTTCCTTCCTCTAAATGACCAAGATGCCGACATCGGC
CACCTCTTCCGTGCTGCGTTCTTCATCGATGCCAGTACCACGATATCCGTTGTTGAAAGTTTTAACTTCTTTAGTTATATGTTCAGCGTTCATTATTTAACAGCGTTTGGTCGGCCGCCGGCATTCTTACCCGCCCCACCCGGGTAGGTCCTACAGGGTCGTAGGTCTTCCAGGGTGAGCGCGACCTGCCGAG

CGCCATCATACCGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGATTCATTTTTGTTTTGTTTTCTCAGCGAGGTACTATACAAACAAGAGTTTAGAAGGGGGCCACCGTCAGGCCTGCTCCTGCCCCCAGGGGCGGGGGCGGGGAGGCCTAGGTCCTCCAGCCTTGCCGATGCAACGATTTGGGTTATGCGTTCCCATAGGGTATCTGTGTACGCCCTAGGGCTCGTTCCAGCAATTATCCCTCCCCTGGTTCCCCCCCGCTTACCTTGTTACGTCTTTTCCTTCC
CCTGATCATTATGCTGCGTTCTTCATCGATGCCCGCGCCAAGAGATCCGTTGTCGAAAGTTTTGACTTGTTATGTTGACGCTCAGACGACACACGCACAGGCCGGGTTGAGGGGGCCTCCGGCGGGCGCGTCCGGGGGACGCCGGGCGGCCTCGGGAGGGGCCGATCCCGCCAAAGCAACACGGGTGGTAACACCGGGTGGGAGGTCGGCCCCGGAGGGCCCGATCTCGGTAATGATCCTTCCGCTGGTTCACCAACGGCAACCTTGTTACGACTTTTACTTCCCCTAAATGACCCAGATCAGTAGT
CACCTCTCTTCTGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTAACTATTCTATATTACTCATCCCCCATTTTAACTTTGGTTTGTGGTCCTCTCGCAGGCGCGGGCCCGCCAAAGCCCCCCGTCCAAGTACGGGCCTGCCAAAGCAACATAGGTATTGTAAACAAAGGGTGGGAGATTAGCCCGGAGTCTTCTCTCTTTAGTTTTCCTTCCGCAGGTTCACCTACGGAAACCTTGTTACGACTTTTACTTCCCCTAAATGACCAAGGAGGTGAGCTCGGAAGTCCC
CTAATTCTACAGGCTGCGTTCTTCATCGATGCCAGGACCAAGAGATCCGTTGTTGAAAGTTTTGATTCATTTTGTA

CGTGATCAGACGGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTAATAAATTTTGTTTTTACTCAGACTACACTAAGAATTTAGATGTCCTTTGGTGGGCACACACTAGCCTAGGCCAGTGTAGAGCCGAAGCCCTACGGCCCACCGAAGCAACAGGGTTGATTAACAAAGGGTTGGAGGTCTTCCCAGATGGGCCTTACTCTTTAATGATCCTTCCGCAGGTTCACCTACGGAAACCTTGTTACGACTTTTACTTCCTCTAACTTACCAATATCACGAGATCGGCCA
CCACATCTATCCGCTGCGTTCTTCATCGATGCCAGAACCAAGAGATCCGTTGTTGAAAGTTTTGGTTTGATTTGTTTAATCAATAAAAATAGATTCAGACGACACGGTTTTGTTCAAGAGTTTTGTGGCTGGCCGTCGGCGGACGCTCTCCAGGGCGGCTGGGCCGACCCCGGCGGTCGTTCGAACCAGTTGGTCGTCCACCGGTCCGCCTAACCAACGACCAGGTAGTGTAGACACACGGTTTGGAGGTCGGGCGCCTTGGGTTCTACCCTTACTCGGTACTGATCCTTCCTCATTTTCCCCTACC
CGGCGCTCTTCTGCTGCGTTCTTCATCGATGCCAGAGCCAAGAGATCCGTTGTTGAGAGTTTTTGCTGTTCGCAATGTGCTCTGACCGCATTAACGTTAGGGTTCAAGGTCCTGCGCCGGGCGCGGGGCCCGGCGAGGCAACAGGTGTGTGGTAGGCAGGTAGCGGGAGGTCTGCCCCCCTGCGAGGGGGGGCCACCTTCTCTCTAATGATCCCTCCGCAGGTTCACCTACGGAGCCCTTGTTACGACTTTTACTTCCTCTACATGACCAAGGCCCCCCGCTCGGCAGCCCCGTTCCTCCGGCATCC
CTTCTATTATCCGCTGCGTTCTTCATCGATGCCAGAGCCCAGAGATCCATTTTTGAAAGTTGTGACTTTCTTTTAC

CGGCGCTCTTCTGCTGCGTTCTTCATCGATGCCAGAGCCAAGAGATCCGTTGTTGAAAGTTTTTGCTGTTCGCAATGTGCTCTGACCGCATTAACGTTAGGGTTCAAGGTCCTGCGCCGGGCGCGGGGCCCGGCGAGGCAACAGGTGTGTGGTAGGCAGGTAGCGGGAGGTCTGCCCCCCTGCGAGGGGGGGCCACCTTCTCTCTAATGATCCCTCCGCAGGTTCACCTACGGATACCTTGTTACGACTTTTACTTCCTCTAAATGACCACGGCTCCCAGCTCGGATGATCGGTTCAGCAGGCATGC
CCTGATCTATCCGCTGCGTTCTTCATCGATGCCGGAACCAAGAGATCCCTTGTTGATAGTTTTGCCTGATTTTCTGTTCCGCCTCCGCCTTCATCCCTCTCTGTCATTACGTTCAGTAGTCCCCCGCCGCTCTCCCCCTCGTGGCTTCCCCGCCTACGCTCCCGTGTTAGTTCTTCTCGGGTGTGAGCTTTGTCTCCCTTAGGCTCCCCCCTCTCAGTCATCCTCCTTCCGCTTGTTCCCCTCCTTACTCCTTCTTATGACTTTTACTTCCTCTTAATTCCACCGATCCTTCCCTCTGACTAGCCGT


Looks right, and for ITS1f? These primer sections of the reads are really degenerate, for some reason. So we put a lot of wildcards into the primer sequence when we search for it:

In [72]:
grep CTTGGT..TTT..A.G.A.TAA <(head -n 10000 rearranged_leafR2.1.fastq)

CTTCCTTCTTCTCTTGGTNCTTTCTATGCACTAAAAGTCGTAACACTGTTTCCGTATTTGAACCTTCTGATGTCTCCTTACCTTTTTCTGTTCCTCTGGCCCTCCCTTTCCTCACTCCTGATTTCACTCCCTTTCGTTCTGCCGTTCGCGGTCCCTTTCCCCCCCGCTCGTTTTGCTTTCACTTACCTTCTTTTCTTGCCCCTCTCTGTCTTTCACTACTTCCCAAAACTTTCCACCCCCTTTCCCTTTTCCCCCCCACCCTCTCTTACCCCCCCCCTACACGCTCTGCCACGCCTCCTTCTTCTCC
CCTCGCTTCCCTCTTGGTNATTTAGAGGAATTAACAGTCGTCACTATGTCTCCGTTTGTGAACCAGCGGTGGGATCATTCTTGTTTTTCCCCTCTACCTCCCTTTGTTTCCCTTCCTTTAACTTTTCCTTCCTCTTGCCTCCTCCCCCTTACCCCCCCCGCCTCCCCCCCCCCGACGGTCCGGCCCCCCCTCGCCTCCACTCCCCCTCTGTTTTCCCCTCCTTTCTTCTGACTACTACCTTCTTATACTCACTCCTTTCACCACCCCTCCTCTTTGTTCTTCCTCCTCTCACTACCCCCCCCGTCCC
CCTGATTTCAAACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACCAGGTCTCCGTTGGTGAACCAGCGGAGGGATCCTTATCGAGTTACCACTCTATAACCCTTTGTGAACATACCTCCATGTTGCTTCTGCGGTCGGCCCCCCTTCCCCCCGCCCCCGCTCCCGCGGGTCGTCCGCCGCAGGTTCACCACCCTCTGTTTTCCCGCCGTTTCTTCTGCTTGGCCCCTCCAACTAATCAAACCTTTTACCCCCCGTTCTCTTCGTTCTTGCCTCCTTCAACCCCCCCGCCTTTCTTCTTCCGCATCAC
CCCCATCATACCCTTGGTNATTTCGAGGCAGTAAAAGTCGTAACCATTTCTCCGTTTGTTAACCAGCGGAGGGCTC

CCCCATTTATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGACCCAGCGGAGGGATCATTGCTTGACCCCAGCGTAAGCTGGGGAATTGCATCACCCAAATTGACCTATCCTTTGTTTTCCTCGGTTGTCGGCTCTGCTTAGCCCCTGGCCCCCATACGGCGCTCACCTTTGGACCAGTCTTGTTTGTATCTGTACTCATTAATCGTCAGTCCCTACTTTCAACAATGTCTCTCTTTCCTCTCGCACCCATTTCTATTCCTGCTGCTCTCTCTTTCTCTCCCTCCTTC
CACCTCTCTTCTCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAATGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACATTATATTGTTATGCGTTGTCTAGTGTTTTGCTTAGCACTTTCACTTACCAACCTAACTATATATACACCCATGTTTTTTCAGTACCTCCCGTTTCCTCGGCAGGCCCGCCTGTCCGTAGGACCACTATTAAACCTATTTTTACTTTAGTCTGCGTCAGCATCTATAACTCTTATACCTTTCAACATCGCTTCCCTTTGTTCTTTCATCCTTTACTTACCCTACCA
CCGCATTACTGACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACCATGTCTCCGTAGTTGAACCTGCTGAGGGATCATTACTGAGTGAGGGCTCACGCCCGACCTCCAACCCTTTGTGAACACATCTTTTTTCTTCGGGGTCGACCCTTCCGGCACTTCTTCTCCTGGCGCCCCCGACGTTCTCCACACACTGCTTCTTTTCGTCTCATTTTAAACAACTTAACCCAAACTTTCCCCCACTGCTCTCTTTCTTTTTCCATCTCTCTACCACCCTCCCCCCTTTAATCCTCAATCTCTTCTTTCTGCC
CTCAAATATGAGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTAGGTGAACCTGCGGAGGGATC

CGTGATTGCCCGCTTGGTNATTTAGAGGCAGTAAAAGTCGTATCCCTGTCTCCGTAGGTTAACCTGCGGCGGGATCCTTCCTGAGTGAGGGCTCACGCCCGACCTCCAACCCTTTGTGAACACCTCTTGTTGCTTCGGGGGCGACCCTGCCGGCACTTCGTCGCCGGGCGCCCCCGAAGGTCTCCACACACTGCATCTTTGCGTCTGCGTTTACACCAATTACCCAACCCTTTCCACATCGGATCTCTTTGTTCTGCCTTCCATGACGTCCTCCTCCGGGCACGATCGGCACACCCTCCTTTTGGTC
CCTGATCTATCTCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTACCGAGTTTCAACACTCCCACCACCATCGTGAACCCCCCTACCGTTGCCTCGGCGGGCGGCCGGCCTACCCTCCCCAGCGCCTGCCTCGCCTTCTTTCCCCCTTCCCCCCTGGCCCCGGCTCCCCCCGCCCCCGTCCCCCTACCCTCCTTCCCCACTCCTCCCCCTCTTACCTCTCCACCTCCCTCCACACTTCCCCCCCTTGTCTCTCTTTTCTTTCCCACCTCTAC
CATGAGTATCACCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAATGTTTCCGTAGGTGAACCTGCGGAAGGATCATAAAGACGCTTCGGCTCTAGCCGGAGCGCACTTCCACACTGAGTATAACTTCTAGGATAATACATATACACAATCAGATTATTTAAATCTCCTCCCACCGTATCTCTTGGCTCTCGCCTCTCTTAATAACGCAGCGTTATACGTTCGGCAGAGCGTCCTTTAGGCACAGCGTTTGTTTCTCGCTTCTCTCCGTTTCCTTACCCCTATCCTCCTTTCCTCCGCTCTACCTT
CCGCATCGCAGACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC

CTCAAACTCAAACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTACTGAGTTTACGCTCTACCACCCTTTGTGTACCTACCTATAACTGTTTCTTCGGCGGGTAGGGTCTCCCTTACCCTCCCGGCCTCCCGCCCCCGGGCGGGTCGGCCCCCGCCGGCCGCTATCCAAACCCTGATTTACCCACGTTTCTTCTGAGTTGTACCAGCAAATAATCAAACCTTTTTCCACCCGATCTCTTTGTTCCGGCCTCGATTATATAAGCACCTTTTCT
CACCTCCTACAGCTTGGTNATTTAGAGGCAGTAAAAGTCGTAACAAGGTCTCCGTAGGTGCACCTGCGGAGGGATCATTAGAGAGAAGGTGGCCCCCCCTCGCAGGGGGGCCGACCTCCCGCTACCTGCCTACCACACCCCTGTTGCCTCGCCGGGCCCCGCGCCCGGCCCCTGACCTTGACCCCTACCGTTACTTCGGTCAGTGCCCCTTTCGACCCCCCAACACTTTCCACCACGCATCTCTTTTCCCTGTCATCCTTCACTACCGCACCCTCTTGCTTTCCGCTTCTCCTCCTCTTCTTCCATC
CGCCATCTACAGCTTGGTNATTTCGAGGCAGTAAAAGTCGTAACACGGTCTCCGTAGGTGAACCTGCGGAGGGATCCTTACCGAGTGAGGTCTCTCCCCCTACCTCCCTCCCTGTGTGAACGCCATTCTCGTTTCTTCCGGTCCCACCCTTCCTTTGCGCCCCCGCCTCTCCTCTCACCTCTGCCTCCCTTCTTCCGCTTCCAATTTACTCCAACCACACTTTCACCACCTCATCTCTTTTTTCTCGCACCTTTCTCGAACCCACCCTTTACCCCTTGTCTCATCATCGTCTCTTTCCCATCCTTTT
CGTTAACATACCCTTGGTNATTTAGAGGAAGTAAATGTTGCACTTTATTCAAAACAAAAGTACAAATATGCAAAGC

CGTGATCCCGTACTTGGTNATTTAGAGGAAGTAACAAATGACAAGAACGAAGTACGCATACATCAAAGGAGTACAATAATGGCATAAACGTTAAGACCACCGAGCATCGCTGAAGCACGCAGCTACGGCCGATCGGAAGAGCGTCGTGTAGGGAAAGCGTGTAGCTCTCGGTGGTTGCCTCACTCTTATCATCTTACATGTTTCTTACTTTTCACTTTCTCTCCTCCCTTCCTCTCCTTCTTCTCTCCTTTCTCTCCCTTTACCCTCCTCTTCTCCTTTCCTCTCCCTTCTCTCTTCCCTTTTCTCT
CGTGATTATCACCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAATTTCTCCGTTGTTGAACCAGCGGATGGATCATTAGGACTCTCAAGAGTTGTAAACCACTGTTAACTTACCTACCCGTTGCCTCGGCGTGTGCCCCGTTTCGCCGCGGCGCTCGCCCGCCGTTTGCCCCCCTCCCTCTCTCTGCCTTGCCTTCCCCAGCAACTTCACTCTCTTTATCACTTTCACCACCCTCTCTCTTCGCTCTTCTCTCCATGACAACCCCACCTTCTTCACACCTCTACTGCCTCCTCTCGTGTATCATC
CTCAAATTATTTCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTAGGTGAACCTGCGGAGGGATCATTACTGAGTGCGGGCTCACGCCCGACCTCCAACCCTTTGTGAACCACTTGTTGCTTCGTGTGCTACCCGGCCGTTTCGGCCGTCCGCGGCCCCCCCGGCGGCCTTCAAACCCTGCATCTTTGCGTCCGAGTTTAACATCAATTACACAAAACTTTCCACAACGGATCTCTTTGTTCTGGCCTCCCTTAACAACCCCTCCCATCCCCACCGCACTATCCTCCTTTACGGCC
CCTGATTTCAAACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAATGTCTCCGTTGGTGAACCAGCGGAGGGATC

...

CTCAAATTCAAACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAAGCAGCGGAGGGATCATTACAGGAACTCGCAAGAGCTGTAAACCACTGTGAACCTACCTATAACGTTGCCTCGGCGGGTGCCCCGGGTCCCCCGGGGAGCCCGCCCGCCTGCCGCCCCTTACTCTATCCCTGCGTTGGCCTCCCCAGACTAATACAAGCAAGCTAAAACCTTCAACAACGGACCCCTGTGCCCGGGCATCCATCAAGACCGCAGCCTTTAACACTAGGACAGCCTACATGGACAGG
CCACATATATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTAGAAGAATAACTCTCTTCAAAACCTGTGGACCTATACCTGTTGTACTCTACGCCGTTCTTTCGGCGGGCTTACCGGACCTCCGGGGCATCCCTTCGGGGAGCTTTGGCGCTCAGGTGCCCGCCGGGGGCGGCGAATCAAACCCTTATTATATTTTCAGGCCTCTGTGATTAAAAACAAATAAGTCAAAACTTTCAAAAATGGGACTCTTTGGTCTGGCCCCGAATAAA
AGTTAAGTATTTCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTAGGTGAACCTGCGGAGGGATCATTACCGAGCGAGGGCTCACGCCCGACCTCCAACCCTGTGTGAACGCAAATCTCGTTGCTTCGGGGGCGACCCTGCCGTGGCGCCCCCGGAGGTCATCTCAACTCTGCATCCCTGCGTCGGAGTCAAAGTTAATCAAACAAAACTTTCAACAACGGATCTCTTGGTTCTG
GGGCGCTCTTCTCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTAGGTGAACCTGCGGAGGGATCATTAGAGAGAAGGTGGCCCCCCCTCGCAGGGGGGCAGACCTCCCGCTACCTGCCTACCACA

CGTGATTGCCCGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTAAAGAGTTCTATAACTCCCGAACCCATGTGAACAATACCTTACGTTGCCTCGGCAGGTCGCGCTCACCCTGTAAGTCTTACCCTGTAAGACCTACCCGGGTGGCGCGGGGGAGCATGCCGGCGGCCTACCAAACTCAGTTTAATATTGAATTCTGAACTTATAAATAAATAAGTAAAAAATTTCAAAAAGGGCACATTTGGTTCTGGCAACGATGAATAACGCAAAGG
ACTGGCGGGGCGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTATAGAGTTTTCTAAACTCCCAACCCATGTGAACTTACCTTTTGTTGCCTCGGCAGAGGTTACCTGGTACCTGGAGACAGGTTACCCTGTAGCAGCTGCCGGTGGACTACTAAACTCTTGTTATTTTATGTAATCTGAGCGTCTTATTTTATTAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGCATCGATGAAAAACGCAGACGCCCCAGACCGGAAGAGGG
ACTGATAATGAGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTACCGAGCTGCGCTCCCCGCACTCCCAAACCCCCCAGCGCGAACCAATACCATCACGGTTGCCTCGGCGGGCCGCTCGAGCCCGACGCAGGGAACGCCCAGCAACTCCAGCGTCGGAACGGCGCGCGACGGCCGCGGGCACGCAGCGAGAGAGCGCGGGCGCACCGGTATGGGACAGGCCACCGGCCACGGACGGAGAAGGAGAACAAAGAAAAAAAAGACCACAACGA
CCACATATATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC

CGTGATCGCCCGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTACTGAGTTTACGCTCTACAACCCTTTGTGAACATACCTATAACTGTTGCTTCGGCGGGCAGGGTCTCCGTGACCCTCCCGGCCTCCCGCCCCCGGGCGGGTCGGCGCCCGCCGGAGGATAACCAAACTCTGATCTAACGCAGTTTCTTCTGAGTGGTCAGAGCAACTAAACCAAACTTTTAAAAACGGATCCTTTCGTTCTGGCCTCGCAGACAAAACCAGCCGGGCG
ACCCATATATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTGCTTGACCCCAGCGTAAGCTGGGGAATTGCATCACACAAATTGACCTATCCTTTGTTTGCCTCGGTGGGCGGCTCAGCTGAGCCCATGGACCCGAAAGGGCGCTCACCGTTGGACCAGTCTTGTTTGAATCTGTACTCAGTAAAAGAAAGTCACAACTTTCAACAATGGATCTCTTGGCGCTGGCAACGCTGAGGAACGCAGCGGATATACACAGGAAAAGCGAACGG
ACCCATTTATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTGCTTGACCCCAGCGTAAGCTGGGGAATTGCATCACACAAATTGACCTATCCTTTGTTTGCCTCGGTGGGCGGCTCAGCTGTGCCACTGGACCCGACAGGGCGCTCACCGTTGGACCAGTCTTGTTTGCATCAGTCCTCACTAAAAGAACGTCCAACTTTCCACCAATGGACCTCTTGGCCCGTGCACCCCTGCATCACGACGCGGATTACGACCGGCAGGCCGTCGTG
CACCTCCTACAGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTAGGTGAACCTGCGGAGGGATC

CACCTCGAAATGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTATCGAGTTATTCAACTCCCAAACCCTATGTGAACTTACCACTTGTTGCTTCGGCGCGCGCTTCGGTGCCGCCGAGGACACTTAAACTCTTTGTTTTTATAAGCATTTCTGAGTGGCCGAAAGGCAAACATACAAATGAATCAAAACTTTCAACAACGGATCTCTTGGTTCTGGCATCGATGAAGAAAGCAGCCATTTAAAATCGGAAGAGCGTCCGTGAGGGACAGAG
CGTGATTGCCCGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGGTGAGTAGTCCTCGGCGAACGTCGGACGATCAGTAACTAACCATTCACAGTGAACCTGCGGAAGGAACATTAAAGAGAACGGGCCCGCAAGGGCACGACCTCCCACCCTCCGTAGCCGCCAATCGCCTTGCCTCGGCCGGCCGCCCCCCAGGGGGGAGCGCAGGCGCCCGCCCCGGCAGCCCTCGCCCCGTGGCCGGCGCACCCGCGCCGCGCCCCCACAGTCCGTGCCCCCTCGCTCATTGATACAAA
CACCTCAACTGACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTAAAGAGTATGGGTTCCGCAAGGGCCCAATCTCCCACCCTCTGTTGTCGCGTAAACATGTTGCTTCGGCGGGCCCGAGCCTCGCGCCGGCCGAGTCTCCAAGCGAGCCCGGACAGCGCCCGCCGCGGGAAACAACCAACTCTGTCTGACGCGCCAGTCTGAGTAAAAATCTTTAAAAAATAAAAAATTTCAACAACGGATCACTTGGTTCTGGCCACGTTGAAGAACGC
ATAATTAAGACGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATC

ACTGGCGAACCACTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTACTGAAATACGTAATTCTCTTGAAAGGTAATCGAGTACGCCGGCCCTAAAAAGCCGGCGGAAAGAGCCCTTCCCACCCTTGTGTACCTTACCATGTTGCTTTGGCGGGCCGACCCGGTTTCGACCCGGGCGGCCGGCGCCCCCAGCCTTCACCGGCCCGGACGCCCGGCTACGTGCCTGCCCGTATACAAACCTCACGCGACCATTTCGCTAAGGCCCGACTTTATAT
ACTGATCTATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATCATTACCGAGTTGCAAAACTCCCCCAACCATCGTGAACCCACCTACCGTTGACTCGGCGGGCGGCAGGCATACCCCCACCAGGGTCTGCTACGCCGTCTGTCCCCCTGCCCCCCGGGGCTCGGGGGCGCCCGCCGGCGGACCCCTCAACTAAGACCCCAGGGCACCCCAGAGGACCGTACGCACAACAGAAAAACTTTCAAAAAGGAACATCCTGTTTATGGCCTATGTGAA
CACCTCAAGACGCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTAAAGAGTATGGGTTCCGCAAGGGCCCAATCTCCCACCCTCTGTTGTCGCGTAAACATGTTGCTTCGGCGGGCCCGAGCCTCGCGCCGGCCGAGTCTCCAAGCGAGCCCGGACAGCGACCGCCGCGGGAAACAACCAACCCTGTCCGCAGCGCCAGTCTGAGTAAAAATACAAAAAAAACACAAAATTTCAAACACCGAACTCTTGGTTACGGAGTAGACTAAACAAGA
CCACATATATCCCTTGGTNATTTAGAGGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC

Looks good. Clip this using [fastx_trimmer](http://hannonlab.cshl.edu/fastx_toolkit/commandline.html#fastx_trimmer_usage), which can handle fastq files:

In [73]:
fastx_trimmer -i rearranged_leafR1.1.fastq -f 33 -o rearranged_leafR1.2.fastq

fastx_trimmer: Error: invalid quality score data on line 4 (quality_tok = "CF8C+BFBFEF9EF,,;<@C,,-,;C,,,,,,;CC,CC,BE,,,,,<CFF,,;CC,EE############################################################################################################################################################################################################################################"


: 1

<h3>Order of reads</h3>